# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [16]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [17]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'sk_541d349ea9ee429782bf1e5562bf74fb'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [18]:
symbol='AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2913322239180,
 'week52high': 198.57,
 'week52low': 163.63,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 164.08,
 'week52change': 0.08298433751216283,
 'sharesOutstanding': 15334082000,
 'float': 0,
 'avg10Volume': 43488696,
 'avg30Volume': 61216534,
 'day200MovingAvg': 182.36,
 'day50MovingAvg': 178.25,
 'employees': 164000,
 'ttmEPS': 6.43,
 'ttmDividendRate': 0.966204324874062,
 'dividendYield': 0.005085553581104595,
 'nextDividendDate': '',
 'exDividendDate': '2024-05-10',
 'nextEarningsDate': '2024-08-01',
 'peRatio': 29.02033329528136,
 'beta': 1.0670490915928768,
 'maxChangePercent': 73.65225933202358,
 'year5ChangePercent': 3.447196129331271,
 'year2ChangePercent': 0.29245777182157706,
 'year1ChangePercent': 0.09452845232443984,
 'ytdChangePercent': -0.007988203836781449,
 'month6ChangePercent': 0.003109804066504296,
 'month3ChangePercent': 0.050080418285625816,
 'month1ChangePercent': 0.12525452820332417,
 'day30

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [19]:
data['year1ChangePercent']

0.09452845232443984

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [20]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
    
my_columns = ['Ticker', 'companyName', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [21]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_series = pd.Series([symbol, data[symbol]['stats']['companyName'], data[symbol]['quote']['latestPrice'], data[symbol]['stats']['year1ChangePercent'], 'N/A'], index = my_columns)
        new_row_df = new_series.to_frame().T
        final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index=True)
final_dataframe

,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,A,Agilent Technologies Inc.,145.74,0.239713,N/A
1,AAL,American Airlines Group Inc,11.49,-0.063415,N/A
2,AAP,Advance Auto Parts Inc,64.435,-0.364297,N/A
3,AAPL,Apple Inc,191.39,0.094528,N/A
4,ABBV,Abbvie Inc,154.59,0.175091,N/A
...,...,...,...,...,...
490,YUM,Yum Brands Inc.,134.795,0.066577,N/A
491,ZBH,Zimmer Biomet Holdings Inc,116.56,-0.081634,N/A
492,ZBRA,Zebra Technologies Corp.,318.86,0.232396,N/A
493,ZION,Zions Bancorporation N.A,41.325,0.563741,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [22]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,N/A
1,NVDA,NVIDIA Corp,1138,1.925664,N/A
2,GPS,"Gap, Inc.",21.175,1.551276,N/A
3,NRG,NRG Energy Inc.,83.09,1.551207,N/A
4,SLG,SL Green Realty Corp.,48.87,1.48848,N/A
5,GE,GE Aerospace,165.755,1.068744,N/A
6,QCOM,"Qualcomm, Inc.",210.1,1.0315,N/A
7,TT,Trane Technologies plc,324.52,0.997222,N/A
8,HWM,Howmet Aerospace Inc,84.54,0.966994,N/A
9,WRK,WestRock Co,53.265,0.928586,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [23]:
def portfolio_input():
    global portfolio_size
    portfolio_size = 100000

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

100000


In [24]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\trygv\AppData\Local\Temp\ipykernel_10636\2310163137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])


,Ticker,companyName,Price,One-Year Price Return,Number of Shares to Buy
0,LB,L Brands Inc,79.92,2.281287,24
1,NVDA,NVIDIA Corp,1138,1.925664,1
2,GPS,"Gap, Inc.",21.175,1.551276,92
3,NRG,NRG Energy Inc.,83.09,1.551207,23
4,SLG,SL Green Realty Corp.,48.87,1.48848,40
5,GE,GE Aerospace,165.755,1.068744,11
6,QCOM,"Qualcomm, Inc.",210.1,1.0315,9
7,TT,Trane Technologies plc,324.52,0.997222,6
8,HWM,Howmet Aerospace Inc,84.54,0.966994,23
9,WRK,WestRock Co,53.265,0.928586,36


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [25]:
hqm_columns = [
                'Ticker',
                'Company Name',
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            new_series =pd.Series([symbol, 
                data[symbol]['quote']['companyName'],   
                float(data[symbol]['quote']['latestPrice']),
                'N/A',
                float(data[symbol]['stats']['year1ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month6ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month3ChangePercent']),
                'N/A',
                float(data[symbol]['stats']['month1ChangePercent']),
                'N/A',
                'N/A'
                ], 
                index = hqm_columns)
            new_row_df = new_series.to_frame().T
            hqm_dataframe = pd.concat([hqm_dataframe, new_row_df], ignore_index=True)
        except:
            pass

  
        
hqm_dataframe

,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,145.74,N/A,0.239713,N/A,0.192381,N/A,0.083503,N/A,0.076013,N/A,N/A
1,AAL,American Airlines Group Inc,11.48,N/A,-0.063415,N/A,0.104355,N/A,-0.13066,N/A,-0.0317,N/A,N/A
2,AAP,Advance Auto Parts Inc,64.415,N/A,-0.364297,N/A,0.369008,N/A,0.060009,N/A,-0.06601,N/A,N/A
3,AAPL,Apple Inc,191.39,N/A,0.094528,N/A,0.00311,N/A,0.05008,N/A,0.125255,N/A,N/A
4,ABBV,Abbvie Inc,154.59,N/A,0.175091,N/A,0.146356,N/A,-0.119153,N/A,-0.026814,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,Yum Brands Inc.,134.795,N/A,0.066577,N/A,0.085615,N/A,-0.014952,N/A,-0.036978,N/A,N/A
487,ZBH,Zimmer Biomet Holdings Inc,116.56,N/A,-0.081634,N/A,0.03264,N/A,-0.061075,N/A,-0.025304,N/A,N/A
488,ZBRA,Zebra Technologies Corp. - Class A,318.86,N/A,0.232396,N/A,0.394844,N/A,0.195894,N/A,0.097675,N/A,N/A
489,ZION,Zions Bancorporation N.A,41.325,N/A,0.563741,N/A,0.266856,N/A,0.13744,N/A,0.037207,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [26]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_returns = hqm_dataframe[f'{time_period} Price Return']
        valid_price_returns = [x for x in price_returns if pd.notna(x)]        
        price_return = hqm_dataframe.loc[row, f'{time_period} Price Return']
        if pd.notna(price_return):
            hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(valid_price_returns, price_return)/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.629328
1      0.175153
2      0.020367
3      0.403259
4      0.533605
         ...   
486    0.350305
487     0.14664
488    0.615071
489     0.90224
490    0.315682
Name: One-Year Return Percentile, Length: 491, dtype: object
0      0.688391
1      0.511202
2       0.90224
3      0.252546
4      0.613035
         ...   
486    0.468432
487    0.293279
488     0.92057
489    0.800407
490    0.162933
Name: Six-Month Return Percentile, Length: 491, dtype: object
0      0.716904
1      0.112016
2      0.639511
3      0.596741
4      0.136456
         ...   
486    0.360489
487    0.256619
488    0.928717
489     0.85947
490    0.093686
Name: Three-Month Return Percentile, Length: 491, dtype: object
0       0.85336
1      0.279022
2       0.10387
3        0.9389
4      0.305499
         ...   
486    0.238289
487    0.309572
488    0.900204
489    0.696538
490      0.8778
Name: One-Month Return Percentile, Length: 491, dtype: object


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,145.74,N/A,0.239713,0.629328,0.192381,0.688391,0.083503,0.716904,0.076013,0.85336,N/A
1,AAL,American Airlines Group Inc,11.48,N/A,-0.063415,0.175153,0.104355,0.511202,-0.13066,0.112016,-0.0317,0.279022,N/A
2,AAP,Advance Auto Parts Inc,64.415,N/A,-0.364297,0.020367,0.369008,0.90224,0.060009,0.639511,-0.06601,0.10387,N/A
3,AAPL,Apple Inc,191.39,N/A,0.094528,0.403259,0.00311,0.252546,0.05008,0.596741,0.125255,0.9389,N/A
4,ABBV,Abbvie Inc,154.59,N/A,0.175091,0.533605,0.146356,0.613035,-0.119153,0.136456,-0.026814,0.305499,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,Yum Brands Inc.,134.795,N/A,0.066577,0.350305,0.085615,0.468432,-0.014952,0.360489,-0.036978,0.238289,N/A
487,ZBH,Zimmer Biomet Holdings Inc,116.56,N/A,-0.081634,0.14664,0.03264,0.293279,-0.061075,0.256619,-0.025304,0.309572,N/A
488,ZBRA,Zebra Technologies Corp. - Class A,318.86,N/A,0.232396,0.615071,0.394844,0.92057,0.195894,0.928717,0.097675,0.900204,N/A
489,ZION,Zions Bancorporation N.A,41.325,N/A,0.563741,0.90224,0.266856,0.800407,0.13744,0.85947,0.037207,0.696538,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [27]:
from statistics import mean

print(len(hqm_dataframe.index))
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
(hqm_dataframe)

491


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,145.74,N/A,0.239713,0.629328,0.192381,0.688391,0.083503,0.716904,0.076013,0.85336,0.721996
1,AAL,American Airlines Group Inc,11.48,N/A,-0.063415,0.175153,0.104355,0.511202,-0.13066,0.112016,-0.0317,0.279022,0.269348
2,AAP,Advance Auto Parts Inc,64.415,N/A,-0.364297,0.020367,0.369008,0.90224,0.060009,0.639511,-0.06601,0.10387,0.416497
3,AAPL,Apple Inc,191.39,N/A,0.094528,0.403259,0.00311,0.252546,0.05008,0.596741,0.125255,0.9389,0.547862
4,ABBV,Abbvie Inc,154.59,N/A,0.175091,0.533605,0.146356,0.613035,-0.119153,0.136456,-0.026814,0.305499,0.397149
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,YUM,Yum Brands Inc.,134.795,N/A,0.066577,0.350305,0.085615,0.468432,-0.014952,0.360489,-0.036978,0.238289,0.354379
487,ZBH,Zimmer Biomet Holdings Inc,116.56,N/A,-0.081634,0.14664,0.03264,0.293279,-0.061075,0.256619,-0.025304,0.309572,0.251527
488,ZBRA,Zebra Technologies Corp. - Class A,318.86,N/A,0.232396,0.615071,0.394844,0.92057,0.195894,0.928717,0.097675,0.900204,0.841141
489,ZION,Zions Bancorporation N.A,41.325,N/A,0.563741,0.90224,0.266856,0.800407,0.13744,0.85947,0.037207,0.696538,0.814664


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [28]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
print(hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)['HQM Score'][:51])
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe

334    0.997963
377    0.989817
329    0.989817
225    0.982688
316    0.979124
331    0.971996
267    0.953157
41     0.951629
475    0.947047
466    0.940428
236    0.939919
258    0.936354
277    0.935336
268    0.932281
36     0.928208
434    0.925153
379    0.922607
202    0.921079
191    0.918024
172    0.913442
374    0.908859
28     0.906823
160     0.89613
107    0.892057
142    0.886965
322     0.88442
9      0.881874
463    0.877291
47       0.8722
75     0.869145
275    0.867108
203    0.864053
419    0.856415
410    0.854379
176    0.850305
248    0.846232
230    0.844705
437    0.844196
180    0.842668
488    0.841141
104    0.838086
485     0.83554
370    0.831466
53     0.821283
137    0.820774
489    0.814664
432    0.812627
320    0.805499
198    0.805499
69     0.803462
197    0.802953
Name: HQM Score, dtype: object


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,145.74,N/A,0.239713,0.629328,0.192381,0.688391,0.083503,0.716904,0.076013,0.85336,0.721996
1,AAL,American Airlines Group Inc,11.48,N/A,-0.063415,0.175153,0.104355,0.511202,-0.13066,0.112016,-0.0317,0.279022,0.269348
2,AAP,Advance Auto Parts Inc,64.415,N/A,-0.364297,0.020367,0.369008,0.90224,0.060009,0.639511,-0.06601,0.10387,0.416497
3,AAPL,Apple Inc,191.39,N/A,0.094528,0.403259,0.00311,0.252546,0.05008,0.596741,0.125255,0.9389,0.547862
4,ABBV,Abbvie Inc,154.59,N/A,0.175091,0.533605,0.146356,0.613035,-0.119153,0.136456,-0.026814,0.305499,0.397149
5,ABMD,Abiomed Inc.,381.02,N/A,0.0,0.256619,0.0,0.232179,0.0,0.441955,0.0,0.466395,0.349287
6,ABT,Abbott Laboratories,101.46,N/A,0.01115,0.274949,0.008726,0.260692,-0.146559,0.083503,-0.051892,0.167006,0.196538
7,ACN,Accenture plc - Class A,294.49,N/A,-0.004244,0.236253,-0.097985,0.087576,-0.209367,0.04277,-0.033376,0.262729,0.157332
8,ADBE,Adobe Inc,479.95,N/A,0.151761,0.490835,-0.232449,0.02444,-0.132996,0.10998,0.001822,0.496945,0.28055
9,ADI,Analog Devices Inc.,230.05,N/A,0.373377,0.761711,0.298417,0.841141,0.25503,0.961303,0.155815,0.96334,0.881874


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [29]:
position_size = float(100000) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\trygv\AppData\Local\Temp\ipykernel_10636\434239120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])


,Ticker,Company Name,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,145.74,13,0.239713,0.629328,0.192381,0.688391,0.083503,0.716904,0.076013,0.85336,0.721996
1,AAL,American Airlines Group Inc,11.48,170,-0.063415,0.175153,0.104355,0.511202,-0.13066,0.112016,-0.0317,0.279022,0.269348
2,AAP,Advance Auto Parts Inc,64.415,30,-0.364297,0.020367,0.369008,0.90224,0.060009,0.639511,-0.06601,0.10387,0.416497
3,AAPL,Apple Inc,191.39,10,0.094528,0.403259,0.00311,0.252546,0.05008,0.596741,0.125255,0.9389,0.547862
4,ABBV,Abbvie Inc,154.59,12,0.175091,0.533605,0.146356,0.613035,-0.119153,0.136456,-0.026814,0.305499,0.397149
5,ABMD,Abiomed Inc.,381.02,5,0.0,0.256619,0.0,0.232179,0.0,0.441955,0.0,0.466395,0.349287
6,ABT,Abbott Laboratories,101.46,19,0.01115,0.274949,0.008726,0.260692,-0.146559,0.083503,-0.051892,0.167006,0.196538
7,ACN,Accenture plc - Class A,294.49,6,-0.004244,0.236253,-0.097985,0.087576,-0.209367,0.04277,-0.033376,0.262729,0.157332
8,ADBE,Adobe Inc,479.95,4,0.151761,0.490835,-0.232449,0.02444,-0.132996,0.10998,0.001822,0.496945,0.28055
9,ADI,Analog Devices Inc.,230.05,8,0.373377,0.761711,0.298417,0.841141,0.25503,0.961303,0.155815,0.96334,0.881874


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [30]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: